<a href="https://colab.research.google.com/github/p-ai-org/p-colleges/blob/main/rate_my_professor_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import requests
import statistics 

In [ ]:
# first, let's make a big dataframe with all of the rate my professors data 
urls = [
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_1_to_32.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_33_to_46.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_47_to_51.json",
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_51_to_161.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_1_to_32.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_162_to_300.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_300_to_397.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_398_to_472.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_473_to_631.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_632_to_753.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_754_to_894.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_895_to_1022.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_1023_to_1344.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_1345_to_2357.json", 
    "https://raw.githubusercontent.com/p-ai-org/p-colleges/main/Brian/Rate%20My%20Professor/all_colleges_RMP_2358_to_2940.json"
]

dataframes = []
for url in urls:
  response = requests.get(url)
  data = response.json()
  df = pd.DataFrame(data)
  df = df.T
  dataframes.append(df)

rmp_data_full = pd.concat(dataframes)
rmp_data_full = rmp_data_full.dropna(subset=['Comments'])
display(rmp_data_full.head())


,School_Overall,School_Reputation,School_Location,School_Facilities,School_Food,School_Happiness,School_Opportunities,School_Clubs,School_Safety,School_Social,School_Internet,Comments
Abilene Christian University,3.8,4.0,3.1,3.9,3.0,3.9,3.8,3.6,4.3,3.4,3.6,{'ACU: BEST school ALL professors: care genuin...
Abraham Baldwin Agricultural College,3.6,3.6,3.6,4.0,3.1,3.9,3.5,3.5,4.4,3.3,3.0,{'Loved my time there!': {'Comment_Date': 'Mar...
Academy of Art University,3.1,3.2,3.7,3.1,2.7,3.2,3.2,2.8,3.0,2.8,3.4,{'This school is 100% a scam. The teachers are...
Bryn Athyn College,3.4,3.3,3.6,4.0,2.8,3.6,3.1,2.8,4.8,2.8,2.9,{'Most of the professors will work with you 10...
Adams State University,3.4,3.3,3.2,3.8,2.7,3.9,3.2,3.1,3.7,3.1,3.2,{'Not a great school. Transferred out after on...


In [ ]:
def filter_comments(dict, lower_benchmark, upper_benchmark):
  '''helper function to get a string of comments from the dictionary in rmp_data["Comments"] based on rating'''
  keys_list = list(dict.keys())
  values_list = list(dict.values())

  filtered_comments = ""
  for i in range(len(keys_list)):
    values_dict = values_list[i]
    ratings = list(values_dict.values())[2:]
    median_rating = statistics.median(ratings)
    if median_rating >= lower_benchmark and median_rating <= upper_benchmark: 
      filtered_comments = filtered_comments + keys_list[i]
  
  return filtered_comments 

def filter_df_by_rating(rmp_data, lower_benchmark, upper_benchmark): 
  ''' filters the rmp data based on the range of ratings we want '''
  new_df = rmp_data.copy()
  new_df["Comments String"] = new_df["Comments"].apply(lambda x: filter_comments(x, lower_benchmark, upper_benchmark))
  new_df.reset_index(inplace = True)
  new_df.rename(columns = {'index': 'College'}, inplace = True)
  new_df = new_df[["College", "Comments String"]]
  new_df = new_df[new_df["Comments String"] != ""]
  return new_df 


In [ ]:
five_star_ratings = filter_df_by_rating(rmp_data_full, 4.1, 5.0)

one_star_ratings = filter_df_by_rating(rmp_data_full, 0.0, 1.9)

all_comments_df = filter_df_by_rating(rmp_data_full, 0.0, 5.0)

In [ ]:
five_star_ratings.head()

,College,Comments String
0,Abilene Christian University,ACU: BEST school\nALL professors: care genuine...
1,Abraham Baldwin Agricultural College,Loved my time there!I love this college and it...
2,Academy of Art University,It is a tough school. Most of the professors a...
3,Bryn Athyn College,Great!This college is simply perfect.
4,Adams State University,Great school with accessible professors who ha...


In [ ]:
one_star_ratings.head()

,College,Comments String
0,Abilene Christian University,Worst school ever just run!I am an online stud...
1,Abraham Baldwin Agricultural College,The administration does not care about the stu...
2,Academy of Art University,This school is 100% a scam. The teachers are h...
3,Bryn Athyn College,Small school small social groups that Are all ...
4,Adams State University,Not a great school. Transferred out after one ...


In [ ]:
all_comments_df.head()

,College,Comments String
0,Abilene Christian University,ACU: BEST school\nALL professors: care genuine...
1,Abraham Baldwin Agricultural College,Loved my time there!ABAC is an awesome school....
2,Academy of Art University,This school is 100% a scam. The teachers are h...
3,Bryn Athyn College,Most of the professors will work with you 100%...
4,Adams State University,Not a great school. Transferred out after one ...


In [ ]:
from scipy import spatial
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#import word embedding 
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove.42B.300d.zip

--2023-04-15 22:07:39--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2023-04-15 22:07:40--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2023-04-15 22:07:40--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
# naming word embedding data 
!head -n 5000 glove.42B.300d.txt > top_5000.txt

In [ ]:
# creating a dictionary for the words and embeddings
embeddings = {}
with open('top_5000.txt', 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings[word] = vector
        
words_with_embeddings = set([w for w in embeddings])


In [ ]:
def clean(text):
    # clean description
    text = text.translate(str.maketrans('', '', string.punctuation))
    description = text.lower().strip()
    words = description.split(" ")
    stops = set(stopwords.words('english'))
    
    # filter out stop words
    words = [w for w in words if not w in stops]

    return words

In [ ]:
"""
takes the list of words, finds the word embeddings for each word, and 
finds the average word embedding for the list of words. 
"""

def calculate_reviews_embedding(words):
    words = [w for w in words if (w in words_with_embeddings)]
    
    if len(words) == 0:
        return None
    
    # calculate embedding and return
    return sum([embeddings[w] for w in words])/len(words)

In [ ]:
five_star_ratings["Embedding"] = [calculate_reviews_embedding(str) for str in five_star_ratings["Comments String"]]
five_star_ratings.head()

five_star_dict = pd.Series(five_star_ratings.Embedding.values, index=five_star_ratings.College).to_dict()


In [ ]:
def sum_vectors(list_of_colleges, list_of_words, dict): 
  sum_of_vectors = 0
  for i in list_of_colleges:
    sum_of_vectors += dict[i]

  for j in list_of_words:
    try:
      sum_of_vectors += embeddings[j]
    except:
      print(j + " not found")
  
  return sum_of_vectors

In [ ]:
def recommend(list_of_colleges, list_of_words, dict):
  vector = sum_vectors(list_of_colleges, list_of_words, dict)

  return_dict = {}
  for college in dict.keys():
    try:
      distance = spatial.distance.cosine(dict[college], vector)
      return_dict[college] = distance 
    except:
      pass
  
  sorted_keys = sorted(return_dict, key = return_dict.get)
  for college in list_of_colleges:
    sorted_keys.remove(college)
  
  return sorted_keys[:10]

In [ ]:
recommend(["Arizona State University", "Scripps College"], ["diverse"], five_star_dict)

['The Citadel - The Military College of South Carolina',
 'St. Louis Community College at Florissant Valley',
 'Harvard University',
 'LeHigh Valley College',
 'South Arkansas Community College',
 'Santa Clara University',
 'Mars Hill University',
 'Community College of Allegheny County',
 'Kansas Wesleyan University',
 'Adams State University']